In [ ]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import os
import functions

### 0) Configuração de pré-pesquisa
Aqui será configurado as informaçoes fornecidas para a pesquisa, como a data de inicio e fim da busca, tamanho da visualização e UF(Unidade Federativa) que será pesquisado os discursos.

In [ ]:
#formato de data = {dd, mm, aaaa}
data_inicio = [28, 3, 2022]
data_fim = [28, 4, 2022]
#tamanho = quantidade de linhas de informações por página (limite 1788 pq? não sabemos)
tamanho = 100
current_page = 1
#Estado onde será realizada a pesquisa ('' para todos)
UF = ''

url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
      

In [ ]:
html = requests.get(url)

content = html.content

soup = BeautifulSoup(content, 'html.parser')

In [ ]:
quantidade_de_discursos= []
qpt = 0 #quantidade de paginas totais

for links in soup.find_all('span', attrs={'class':"visualStrong"}):
    quantidade_de_discursos.append(links.contents)
qpt= int(((int((quantidade_de_discursos[len(quantidade_de_discursos)-1][0]).replace('.','')))/tamanho)+1)
qpt
quantidade_de_discursos

In [ ]:

def UrlToSoup(url):
    html = requests.get(url)
    content = html.content
    return BeautifulSoup(content, 'html.parser')

### 1) Criar um data frame com as informações do site

In [ ]:
url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF={UF}&dtInicio={data_inicio[0]}%2F{data_inicio[1]}%2F{data_inicio[2]}&dtFim={data_fim[0]}%2F{data_fim[1]}%2F{data_fim[2]}&txSessao=&listaTipoFala=&listaFaseSessao=&listaTipoInterv=&txAparteante=&txTexto=&txSumario=&txIndexacao=&BasePesq=plenario&CampoOrdenacao=dtSessao&PageSize={tamanho}&TipoOrdenacao=ASC&btnPesqAvan=Pesquisar'

html = requests.get(url)

content = html.content

soup = BeautifulSoup(content, 'html.parser')

print(url)

In [ ]:
data_frame = [None]*qpt
for i in range(qpt):
    current_page = i+1
    url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
    data_frame[i] = pd.read_html(url)


In [ ]:
data_frame[0][0]

### 2) Criar uma lista com todos os links para os discursos

In [ ]:
link_infos= []
soup = []
temp = []
for i in range(qpt):
        current_page = i+1
        url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
        soup.append(UrlToSoup(url))
        for links in soup[i].find_all('a', attrs={'title':"Íntegra do Discurso"}):
            temp.append((links['href']))
        link_infos.append(temp)
        temp = []


In [ ]:
link_infos

In [ ]:
len(link_infos[0])

In [ ]:
linkDiscurso = []
temp = []
for i in range(qpt):
    for item in link_infos[i]:
        text = item.replace('\t','').replace('\n','').replace('\r','').replace(' ','')
        temp.append(("https://www.camara.leg.br/internet/SitaqWeb/"+text))
    linkDiscurso.append(temp)
    temp = []

In [ ]:
len(linkDiscurso[0])

In [ ]:
len(linkDiscurso[0])

In [ ]:
linkDiscurso

In [ ]:
txt = []
for script in linkDiscurso[0]:
    url_script = UrlToSoup(script).find_all("font")
    txt.append((str((url_script[0].contents)) + "+" + str((url_script[1].contents))).replace('<br/>','').replace('<b>', '').replace('</b>', '').replace('\'', ''))
txt


In [ ]:
for i in range(len(txt)):
    if not(os.path.exists(f"./discursos/file_{i}.txt")):
        textfile = open(f"./discursos/file_{i}.txt", "x")
        textfile.close()

In [ ]:
for i in range(len(txt)):

    textfile = open(f"./discursos/file_{i}.txt", "w")
    textfile.write(txt[i] + "\n")
    textfile.close()

In [ ]:
id_list = []
id = []
for i in range(len(linkDiscurso[0])):
    id.append('nI:')
    id.append((re.findall(r'(?<=nuInsercao=)\d+', linkDiscurso[0][i]))[0])
    id.append("nQ:")
    id.append((re.findall(r'(?<=nuQuarto=)\d+', linkDiscurso[0][i]))[0])
    id.append("nS:")
    id.append((re.findall(r'(?<=nuSessao=)\d+', linkDiscurso[0][i]))[0])
    id.append("nO")
    id.append((re.findall(r'(?<=nuOrador=)\d+', linkDiscurso[0][i]))[0])
    id_list.append("".join(id))
    id = []




In [ ]:
id_list

In [ ]:
data_frame[0][0]['Id'] = id_list

In [ ]:
data_frame[0][0]

In [ ]:
for col in data_frame[0][0]:
    print(data_frame[0][0][col].unique())

In [ ]:
data_frame[0][0]['Id'].nunique()